## Strategy backtesting notebook
The following notebook is used to backtest the strategy and generate results to be analyzed later. 

How the data looks:
"17-09":{
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "6-month":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "12-month":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   },
   "max":{
      "mean":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic16",
                  "barrier":8
               },
               "lower":{
                  "indicator":"stochastic32",
                  "barrier":78
               }
            }
         }
      },
      "order":{
         "mean":{
            "single":{
               "upper":{
                  "indicator":"stochastic8",
                  "barrier":7
               },
               "lower":{
                  "indicator":"adx16",
                  "barrier":61
               }
            }
         }
      }
   }
}

### Env variables and imports

In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

import datetime
import os.path
import sys
import uuid
import copy
import json

# import warnings

import pandas as pd
import requests
import tqdm
import backtrader as bt
import pyfolio as pf
from matplotlib import warnings

warnings.filterwarnings("ignore")  # Avoid some noise

In [18]:
import datetime
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import numpy as np

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd
from itertools import product, combinations

# TIMEFRAME = 'daily'
TIMEFRAME = "weekly"

# FRAMEWORK = 'dask'
FRAMEWORK = "pandas"

# PERC_CHANGE_METHOD = 'mean'
PERC_CHANGE_METHOD = "order aware"

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/forward_opt_results.json"
BACKWARD_OPT_RESULTS_JSON = f"results/{TIMEFRAME}/backward_opt_results.json"

### Utilities

In [ ]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


FORWARD_OPT_JSON = get_forward_opt_json()


def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, "r") as f:
        return json.loads(f.read())


BACKWARD_OPT_JSON = get_backward_opt_json()


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False

    if range_val not in BACKWARD_OPT_JSON[key]:
        return False

    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False

    return True

In [20]:
def get_ticker_csv_path(ticker_name, timeframe):
    return (
        f"/home/narboom23/Projects/licenta/data/{timeframe}_tickers/{ticker_name}.csv"
    )

### Strategy
The main strategy class used

In [19]:
class RaynerTeoStrategy(bt.Strategy):
    params = (
        ("timeframe", "daily"),
        ("period", "max"),
        ("forward_opt_func", "mean"),
        ("backward_opt_func", "mean"),
        ("backward_opt_func_option", "single"),
    )

    def log(self, txt, dt=None, doprint=False):
        """Logging function for this strategy"""
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.inds = dict()
        for i, d in enumerate(self.datas):
            self.inds[d] = dict()

            if self.params.timeframe == "daily":
                self.inds[d]["sma180"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=180
                )
                self.inds[d]["sma200"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=200
                )
                self.inds[d]["sma220"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=220
                )
                self.inds[d]["sma240"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=240
                )
                self.inds[d]["sma260"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=260
                )
                self.inds[d]["sma280"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=280
                )

                self.inds[d]["rsi2"] = bt.indicators.RSI(
                    d.close, period=2, safediv=True
                )
                self.inds[d]["rsi4"] = bt.indicators.RSI(
                    d.close, period=4, safediv=True
                )
                self.inds[d]["rsi6"] = bt.indicators.RSI(
                    d.close, period=6, safediv=True
                )
                self.inds[d]["rsi8"] = bt.indicators.RSI(
                    d.close, period=8, safediv=True
                )
                self.inds[d]["rsi10"] = bt.indicators.RSI(
                    d.close, period=10, safediv=True
                )
                self.inds[d]["rsi12"] = bt.indicators.RSI(
                    d.close, period=12, safediv=True
                )
                self.inds[d]["rsi14"] = bt.indicators.RSI(
                    d.close, period=14, safediv=True
                )

                self.inds[d]["adx8"] = bt.indicators.ADX(d, period=8)
                self.inds[d]["adx16"] = bt.indicators.ADX(d, period=16)
                self.inds[d]["adx32"] = bt.indicators.ADX(d, period=32)

                self.inds[d]["ppo8"] = bt.indicators.PPO(
                    d.close, period1=8, period2=self.params.maperiod
                )  # , period_signal=?)
                self.inds[d]["ppo16"] = bt.indicators.PPO(
                    d.close, period1=16, period2=self.params.maperiod
                )  # , period_signal=?)
                self.inds[d]["ppo32"] = bt.indicators.PPO(
                    d.close, period1=32, period2=self.params.maperiod
                )  # , period_signal=?)

                self.inds[d]["stochastic8"] = bt.indicators.Stochastic(
                    d, period=8, safediv=True
                )
                self.inds[d]["stochastic16"] = bt.indicators.Stochastic(
                    d, period=16, safediv=True
                )
                self.inds[d]["stochastic32"] = bt.indicators.Stochastic(
                    d, period=32, safediv=True
                )
            elif self.params.timeframe == "weekly":
                self.inds[d]["sma30"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=30
                )
                self.inds[d]["sma35"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=35
                )
                self.inds[d]["sma40"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=40
                )
                self.inds[d]["sma45"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=45
                )
                self.inds[d]["sma50"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=50
                )
                self.inds[d]["sma55"] = bt.indicators.SimpleMovingAverage(
                    d.close, period=55
                )

                self.inds[d]["rsi2"] = bt.indicators.RSI(
                    d.close, period=2, safediv=True
                )
                self.inds[d]["rsi4"] = bt.indicators.RSI(
                    d.close, period=4, safediv=True
                )
                self.inds[d]["rsi6"] = bt.indicators.RSI(
                    d.close, period=6, safediv=True
                )

                self.inds[d]["adx3"] = bt.indicators.ADX(d, period=3)
                self.inds[d]["adx6"] = bt.indicators.ADX(d, period=6)
                self.inds[d]["adx9"] = bt.indicators.ADX(d, period=9)

                self.inds[d]["ppo3"] = bt.indicators.PPO(
                    d.close, period1=3, period2=self.params.maperiod
                )  # , period_signal=?)
                self.inds[d]["ppo6"] = bt.indicators.PPO(
                    d.close, period1=6, period2=self.params.maperiod
                )  # , period_signal=?)
                self.inds[d]["ppo9"] = bt.indicators.PPO(
                    d.close, period1=9, period2=self.params.maperiod
                )  # , period_signal=?)

                self.inds[d]["stochastic3"] = bt.indicators.Stochastic(
                    d, period=3, safediv=True
                )
                self.inds[d]["stochastic6"] = bt.indicators.Stochastic(
                    d, period=6, safediv=True
                )
                self.inds[d]["stochastic9"] = bt.indicators.Stochastic(
                    d, period=9, safediv=True
                )

            self.inds[d]["order_placed_days_ago"] = 0

    def next(self):
        optimal_params = get_forward_optimal_params(date, self.params.period)  # TODO
        optimal_sma = optimal_params["sma"]
        optimal_rsi_open = optimal_params["rsi_open_period"]
        optimal_days_ago = optimal_params["days_ago"]

        backward_optimal_params = get_backward_optimal_params(
            date, self.params.period
        )  # TODO

        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size

            if not pos:
                if (
                    d.close[0] > self.inds[d][optimal_sma][0]
                    and self.inds[d][optimal_rsi_open][0] < 30
                ):
                    self.buy(data=d)
            else:
                if self.params.backward_opt_func == None:
                    if (
                        self.inds[d][optimal_rsi_open][0]
                        >= self.params.rsi_close_period
                        or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago
                    ):
                        self.sell(data=d)
                        self.inds[d]["order_placed_days_ago"] = 0
                    else:
                        self.inds[d]["order_placed_days_ago"] += 1
                else:
                    barrier = backward_optimal_params[
                        self.params.backward_opt_func_option
                    ]["barrier"]
                    indicator = backward_optimal_params[
                        self.params.backward_opt_func_option
                    ]["indicator"]

                    if self.params.backward_opt_func_option == "lower":
                        new_condition = self.inds[d][indicator] <= barrier
                    elif self.params.backward_opt_func_option == "upper":
                        new_condition = self.inds[d][indicator] >= barrier
                    elif self.params.backward_opt_func_option == "best":
                        best_type = backward_optimal_params[
                            self.params.backward_opt_func_option
                        ]["indicator"]

                        if best_type == "lower":
                            new_condition = self.inds[d][indicator] <= barrier
                        elif best_type == "upper":
                            new_condition = self.inds[d][indicator] >= barrier

                    if new_condition is True:
                        if (
                            self.inds[d][optimal_rsi_open][0]
                            >= self.params.rsi_close_period
                            or self.inds[d]["order_placed_days_ago"] >= optimal_days_ago
                        ):
                            self.sell(data=d)
                            self.inds[d]["order_placed_days_ago"] = 0
                        else:
                            self.inds[d]["order_placed_days_ago"] += 1
                    else:
                        self.inds[d]["order_placed_days_ago"] += 1

    def stop(self):
        self.log(
            f"(timeframe {self.params.timeframe}, "
            f"period {self.params.period}, "
            f"forward opt func {self.params.forward_opt_func}, "
            f"backward opt function {self.params.backward_opt_func}"
            f"backward opt function option {self.params.backward_opt_func_option}) "
            f"Ending Value {self.broker.getvalue()}",
            doprint=True,
        )

## Run configuration

In [ ]:
DEFAULT_FROM_DATE = datetime.datetime(2004, 1, 1)
DEFAULT_TO_DATE = datetime.datetime(2020, 12, 31)
DEFAULT_CASH = 100.0
DEFAULT_COMMISION = 0.0
DEFAULT_CPU_COUNT = 4
DEFAULT_TICKER_LIST = []

In [19]:
timeframes = [
    "weekly",
    #     'daily' # TODO
]
range_values = [
    "6-month",
    "12-month",
    "max",
]
forward_func_values = ["mean", "order"]
backward_func_values = [
    "mean",
    "order",
    None,
    #     'linear', # TODO
    #     'kNN', # TODO
]
backward_func_options = [
    "upper",
    "lower",
    "best"
    #     'double' # TODO
]

In [ ]:
# TODO: for the weekly timeframe
# TODO: run one strategy
# TODO: run one strategy and save it's output
# TODO: run one strategy and save it's output and parse that output in step 6
# TODO: run each of these by hand and save their output so you can parse them in step 6
# TODO: finish running the weekly dataset generation
# TODO: run step 3 and 4 again for the weekly dataset
# TODO: run these again for the weekly dataset

# TODO: for the daily timeframe
# TODO: run step 3
# TODO: run step 4
# TODO: run step 5

# TODO: finish step 6
# TODO: run it for all data together

# TODO: cleanup the notebooks
# comments and steps
# markdown
# black
    # on docker?
# on git

# TODO: update the documentation
# clear steps and my contribution
# considerations and stuff

# TODO: presentation
# bob wants to invest in the stock market
# bob finds strategies but thinks he can do slightly better using his programming skills which he learned in university
# he sets out to test the strategy on a wider market, sp500 stocks
# he uses daily as well as weekly timeframes cuz he's a chad
# first, checks to see if slightly different values are better instead of his default strategy ones
# he uses different functions to optimize these values
# he takes good care to not overfit stuff
# one important thing he tries is walk forward optimizations
# 1:1 doesnt make sense
# 1:3 can result in several periods of time being empty
# 1:6, 1:12, max are much more relevant
# second, he also adds new params to the strategy
# he finds several oscilating indicators he considers relevant
# for each of these he tracks them for several periods depending on the timeframe
# he is not an expert stock trader so he tries to find patterns automatically
# using lower, upper and best overall barriers and indicators pairs
# only one parameters is used to set a rule because he's already concerned about overfitting
# last, he thinks that the amount of money he invests in any particular trade is relevant as well for the performance of the strategy
# he decides to test 3, 5, 10, 15, 20, 25 percent of his capital on every single trade in turn
# this means that he will either hit more trades or hit fewer but with better performance
# this is already considered very risky and unwise so he doesn't use anything bigger than 25
# he runs his strategy and compiles the results
# looks at the total, cumulative gain
# looks at max drawdown and several ratios
# concludes that investing is risky business which should be done by professionals but also sees the potential algorithmic trading has


PERC_ALLOCATED = 25
# PERC_ALLOCATED = 20
# PERC_ALLOCATED = 15
# PERC_ALLOCATED = 10
# PERC_ALLOCATED = 5
# PERC_ALLOCATED = 3

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro(stdstats=False)

cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)


# Add a strategy
# strats = cerebro.addstrategy(RaynerTeoStrategy, **{
strats = cerebro.optstrategy(
    RaynerTeoStrategy,
    **{
        "timeframe": timeframes,
        "period": range_values,
        "forward_opt_func": forward_func_values,
        "backward_opt_func": backward_func_values,
        "backward_opt_func_option": backward_func_options,
    },
)


datalist = [
    (get_ticker_csv_path(ticker_name, TIMEFRAME), ticker_name)
    for ticker_name in DEFAULT_TICKER_LIST
]

for i in range(len(DEFAULT_TICKER_LIST)):
    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datalist[i][0],
        name=datalist[i][1],
        # Do not pass values before this date
        fromdate=DEFAULT_FROM_DATE,
        # Do not pass values before this date
        todate=DEFAULT_TO_DATE,
        # Do not pass values after this date
        reverse=False,
    )
    data.plotinfo.plot = False

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(DEFAULT_CASH)

# Add pyfolio analyzer for stats
cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.PercentSizer, percents=PERC_ALLOCATED)

# Set the commission
cerebro.broker.setcommission(commission=DEFAULT_COMMISION)

# Run over everything
strat = cerebro.run(maxcpus=DEFAULT_CPU_COUNT)

# Print out the final result
print("Final Portfolio Value: %.2f" % cerebro.broker.getvalue())

In [ ]:
pyfoliozer = strat[1][0].analyzers.getbyname("pyfolio")
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()